Wordnet supersense: Per ogni sinset abbiamo un supersenso associato (esempi su slide)
Utilizzare i supersense dei synset per determinare il semantic type - nltk

I supersense di wordenet non sono il massimo, vediamo alcune alternative:
 * CSI - Ai supersense ci sono associati delle categorie che possono essere utilizzati come supersense

# Esercizio
- Scegliere un verbo transitivo
- Trovare un corpus con > 1000 frasi in cui comprare un verbo scelto (usare un verbo comune)
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

link to resource: https://sentence.yourdictionary.com/kill

In [76]:
f = open("../sentence_cook.txt", "r").read()
print(f)

Get out



here before I kill you.





You can kill me.





This was one way to kill two birds with one stone.





Then you shouldn't kill any of them, if they can be saved!





They didn't enter war to satisfy a desire to kill and maim but to be victorious in the way their society rewarded.





I only kill bad guys.





An old Cat was in a fair way to kill all the Mice in the barn.





You can't just kill people!





I'd use the wrong towel or leave a dish out, and then you'd be threatening to kill me again.





You cannot sell to someone the right to kill you or hold you prisoner.





Hm.... If you wish to kill him, to kill him outright, you can see him...





He thought that I should kill him.





Why should I kill him?





To kill some time I telephoned Ethel Reagan, ostensibly to thank her.





I kill bad guys.





You won't kill Jonny?





He's going to kill me anyway.





American ethanol policies do not "kill" the poor, but they do drive up corn prices.





Hu